In [4]:
!pip install tenso

In [1]:
import cv2
import pickle
import numpy as np
import pandas as pd
from pathlib import Path
import os.path
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image, ImageTk

print(tf.__version__)

# Ensure the TensorFlow version matches the version used to save the model
# Load the pre-trained model
model = tf.keras.models.load_model('FV8.hdf5')

# List of class names (update this list according to your dataset)
class_names = ['dal','idli','kachori','lassi', 'masaladosa', 'pomergranate']  # Example list

# Load a new image for prediction
new_image_path = 'C:/Users/Kartik/Desktop/newdataset/test_images/test_images/apple/106_100.jpg'
image = Image.open(new_image_path)

# Preprocess the image
img_array = np.array(image.resize((299, 299)))
img_4d = img_array.reshape(-1, 299, 299, 3)

# Predict the class of the image
prediction = model.predict(img_4d)[0]
max_prob_index = prediction.argmax()

# Get the class name
predicted_class = class_names[max_prob_index]

print(f'The image belongs to class: {predicted_class}')


2.12.0
1/1 [==============================] - 2s 2s/step
The image belongs to class: lassi


In [ ]:
import cv2
import tkinter as tk
from tkinter import Label, Button, PhotoImage
from PIL import Image, ImageTk
import numpy as np
import pandas as pd
import tensorflow as tf

class ImageClassifierGUI:
    def __init__(self, model_filename='FV5 (1).hdf5', nutrition_filename='FOODNUTRITIONDATABASE.xlsx', glucose_filename='glucose_data.xlsx', window_title='Image Classifier GUI'):
        self.loaded_model = tf.keras.models.load_model(model_filename)
        self.nutrition_data = pd.read_excel(nutrition_filename, index_col='Name')
        self.glucose_data = pd.read_excel(glucose_filename)

        self.root = tk.Tk()
        self.root.title(window_title)

        self.camera = cv2.VideoCapture(0)

        self.label = Label(self.root, text='Press Capture to classify', font=('Helvetica', 16))
        self.label.pack(pady=10)

        self.image_label = Label(self.root)
        self.image_label.pack()

        self.capture_button = Button(self.root, text='Capture', command=self.capture_image, font=('Helvetica', 8))
        self.capture_button.pack(side=tk.LEFT, padx=5, pady=8)

        self.predict_button = Button(self.root, text='Predict', command=self.predict_image, font=('Helvetica', 8))
        self.predict_button.pack(side=tk.LEFT, padx=5, pady=8)

        self.nutrition_button = Button(self.root, text='Nutritional Value', command=self.show_nutritional_value, font=('Helvetica', 8))
        self.nutrition_button.pack(side=tk.LEFT, padx=5, pady=8)

        self.nutrition_label = Label(self.root, text='', font=('Helvetica', 8))
        self.nutrition_label.pack(side=tk.LEFT, padx=5, pady=8)

        self.glucose_label = Label(self.root, text='Select Glucose Level Before Taking Food', font=('Helvetica', 10))
        self.glucose_label.pack(pady=10)

        self.glucose_buttons_frame = tk.Frame(self.root)
        self.glucose_buttons_frame.pack()

        self.glucose_buttons = []
        glucose_ranges = [(80, 100), (101, 120), (121, 140), (141, 160),(161,180),(181,200), (201,float('inf'))]
        for i, (lower, upper) in enumerate(glucose_ranges):
            button = Button(self.glucose_buttons_frame, text=f'{lower}-{upper}', command=lambda l=lower, u=upper: self.set_glucose_level(l, u), font=('Helvetica', 8))
            button.grid(row=0, column=i, padx=5, pady=5)
            self.glucose_buttons.append(button)

        self.after_food_glucose_button = Button(self.root, text='Print Glucose Value After Food', command=self.print_glucose_after_food, font=('Helvetica', 8))
        self.after_food_glucose_button.pack(pady=10)

        self.after_food_glucose_label = Label(self.root, text='', font=('Helvetica', 10))
        self.after_food_glucose_label.pack()

        self.current_prediction = None
        self.glucose_lower = None
        self.glucose_upper = None

    def capture_image(self):
        _, frame = self.camera.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert color from BGR to RGB
        frame = cv2.resize(frame, (500, 500))
        self.current_image = Image.fromarray(frame)
        photo = ImageTk.PhotoImage(image=self.current_image)
        self.image_label.configure(image=photo)
        self.image_label.image = photo

    def preprocess_image(self, image):
        # Resize the image to match the model input size (224x224) and convert to numpy array
        img_array = np.array(image.resize((299, 299)))
        # Convert to float32 and normalize pixel values to [0, 1]
        img_array = img_array.astype('float32') / 255.0
        return img_array

    def show_nutritional_value(self):
        if self.current_prediction is not None:
            if self.current_prediction in self.nutrition_data.index:
                nutrition_values = self.nutrition_data.loc[self.current_prediction]
                nutrition_info = f"Nutritional Values for {self.current_prediction}:\n"
                for nutrient, value in nutrition_values.items():
                    nutrition_info += f"{nutrient}: {value}\n"
                # Display nutritional values in the GUI window
                self.nutrition_label.config(text=nutrition_info)
            else:
                self.nutrition_label.config(text=f'Nutritional data not found for {self.current_prediction}')
        else:
            self.nutrition_label.config(text='Please predict a food item first.')

    def set_glucose_level(self, lower, upper):
        self.glucose_lower = lower
        self.glucose_upper = upper
        self.glucose_label.config(text=f'Selected Glucose Level: {lower}-{upper}')

    def print_glucose_after_food(self):
        if self.current_prediction is not None and self.glucose_lower is not None and self.glucose_upper is not None:
            glucose_value = self.glucose_data[(self.glucose_data['food_item'] == self.current_prediction) & 
                                              (self.glucose_data['before_eating'] == f'{self.glucose_lower}-{self.glucose_upper}')]
            if not glucose_value.empty:
                glucose_value = glucose_value.iloc[0]['after_eating']
                self.after_food_glucose_label.config(text=f'Glucose value after eating {self.current_prediction}: {glucose_value}')
            else:
                self.after_food_glucose_label.config(text=f'No glucose value found for the selected range.')
        else:
            self.after_food_glucose_label.config(text='Please select both food item and glucose level before taking food.')

    def predict_image(self):
        if hasattr(self, 'current_image'):
            img_array = self.preprocess_image(self.current_image)
            img_4d = img_array.reshape(-1, 299, 299, 3)
            # Make prediction
            prediction = self.loaded_model.predict(img_4d)[0]
            prediction_label = ["almond", "apple","banana","beetroot", "burger", "cashew", "cauliflower", "chai",
 "chapati", "cholebhature", "dal", "dates", "fig" ,"icecream","idli"
 , "jalebi","kachori","orange","pineapple","pomergranate","poha", "potato", "raisin", "rice", "tomato", "walnut","watermelon"]
 
            # Get the index of maximum probability
            max_prob_index = prediction.argmax()
            # Get the confidence score for the prediction
            confidence_score = prediction[max_prob_index]
            # Store the predicted class
            self.current_prediction = prediction_label[max_prob_index]
            # Update the label with the prediction and confidence score
            self.label.config(text=f"Prediction: {self.current_prediction} with confidence score {confidence_score}")
        else:
            self.label.config(text="Please capture an image first.")

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    gui = ImageClassifierGUI()
    gui.run()
